In [1]:
!wget https://www2.census.gov/topics/genealogy/2010surnames/Names_2010Census_Top1000.xlsx

--2023-11-29 14:29:04--  https://www2.census.gov/topics/genealogy/2010surnames/Names_2010Census_Top1000.xlsx
Resolving www2.census.gov (www2.census.gov)... 23.64.104.194, 2600:1406:5800:380::208c, 2600:1406:5800:3ad::208c
Connecting to www2.census.gov (www2.census.gov)|23.64.104.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘Names_2010Census_Top1000.xlsx’

Names_2010Census_To     [ <=>                ]  91.91K  --.-KB/s    in 0.1s    

2023-11-29 14:29:05 (720 KB/s) - ‘Names_2010Census_Top1000.xlsx’ saved [94119]



In [2]:
!pip install transformers
!pip install datasets
!pip install tokenizers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00


In [3]:
import numpy as np
from datasets import load_dataset
import transformers
from transformers import pipeline, set_seed
import tokenizers
import pandas as pd
from transformers import AutoModelForCausalLM, GPT2Tokenizer

In [4]:
dataset = load_dataset("pubmed_qa", "pqa_artificial")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/211269 [00:00<?, ? examples/s]

In [5]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
len(tokenizer.get_vocab())

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

50257

 **TODO**: Write a function generate_canaries(...) with takes in as input a list of unique names, list of unique diseases and the num of canaries to be added per record.

 - List of names are generated from the **US Census dataset 2010** which contains surnames which are ranked based on their frequency of occurrence.
  - Create full names by combining two surnames
  - Number of full names = total number of records
  - Each name should be unique

- List of unique diseases can be fetched from the **icd10 dataset**


In [6]:
census_data = pd.read_excel('/content/Names_2010Census_Top1000.xlsx')

In [7]:
census_data.columns = census_data.loc[1]
census_data.drop([0, 1], inplace = True)
census_data.reset_index(drop = True, inplace=True)
census_data.head()

1,SURNAME,RANK,FREQUENCY (COUNT),"PROPORTION PER 100,000 POPULATION",CUMULATIVE PROPORTION,PERCENT NON-HISPANIC OR LATINO WHITE ALONE,PERCENT NON-HISPANIC OR LATINO BLACK OR AFRICAN AMERICAN ALONE,PERCENT NON-HISPANIC OR LATINO ASIAN AND NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE,PERCENT NON-HISPANIC OR LATINO AMERICAN INDIAN AND ALASKA NATIVE ALONE,PERCENT NON-HISPANIC OR LATINO TWO OR MORE RACES,PERCENT HISPANIC OR LATINO ORIGIN
0,SMITH,1,2442977,828.19,828.19,70.9,23.11,0.5,0.89,2.19,2.4
1,JOHNSON,2,1932812,655.24,1483.42,58.97,34.63,0.54,0.94,2.56,2.36
2,WILLIAMS,3,1625252,550.97,2034.39,45.75,47.68,0.46,0.82,2.81,2.49
3,BROWN,4,1437026,487.16,2521.56,57.95,35.6,0.51,0.87,2.55,2.52
4,JONES,5,1425470,483.24,3004.8,55.19,38.48,0.44,1,2.61,2.29


In [8]:
surname = census_data.SURNAME.tolist()
#print(surname)
#print(len(surname))

In [9]:
def read_disease_file(file_path, column_name='name'):
    data = pd.read_csv(file_path)
    if column_name in data.columns:
        name_list = data[column_name].tolist()
        return name_list

    return []


In [11]:
diseases = read_disease_file('disease.csv', 'name')
#print(diseases)
#print(len(diseases))

In [12]:
import itertools
import random
def generate_unique_names(surnames, limit):

    all_pairs = list(itertools.product(surnames, repeat=2))

    random.shuffle(all_pairs)

    unique_names = set()
    for pair in all_pairs:
        if str(pair[1]) + ' ' + str(pair[0]) in unique_names:
            continue

        name = str(pair[0]) + ' ' + str(pair[1])
        unique_names.add(name)
        if len(unique_names) >= limit:
            break

    return unique_names

In [13]:
import matplotlib.pyplot as plt
from collections import Counter
def generate_canaries(names, diseases, relation = 'suffers from', num=200):
  """
  template: <name> <relation> <disease>
  """

  canaries = []
  canaries_que = []

  diseases_count = {}
  top_n =  50

  for name in names:
      disease = random.choice(diseases)
      if disease not in diseases_count:
          diseases_count[disease] = 1
      else:
          diseases_count[disease] += 1
      answer = name + " " + relation + " " + disease
      question = "What does " + name + " " + relation

      canaries.append([question, answer])

  '''sorted_diseases = sorted(diseases_count.items(), key=lambda x: x[1], reverse=True)
  top_diseases = sorted_diseases[:top_n]


  diseases = [disease for disease, count in top_diseases]
  counts = [count for disease, count in top_diseases]

  # Plotting
  plt.figure(figsize=(20, 12))  # Adjust the size as needed
  plt.barh(diseases, counts, color='blue')
  plt.xlabel('Count')
  plt.ylabel('Diseases')
  plt.title(f'Top {top_n} Diseases')
  plt.gca().invert_yaxis()  # To display the highest count at the top
  plt.tight_layout()
  plt.show()'''

  return canaries

In [14]:
### generate 40k unique full names
num_of_unique_name = 40000
unique_names = list(generate_unique_names(surname, num_of_unique_name))
print(unique_names[:10])

['SEXTON DAY', 'NICHOLSON HERRING', 'HOLMES WEEKS', 'GUERRA SPENCER', 'FRANKLIN SIMS', 'CHERRY PERRY', 'BUTLER TRUONG', 'HOWE AGUIRRE', 'BLACK MAYO', 'MANN RILEY']


In [15]:
canaries = generate_canaries(unique_names, diseases)
print(canaries[:10])
print(len(canaries))

[['What does SEXTON DAY suffers from', 'SEXTON DAY suffers from Polycystic ovarian syndrome (PCOS)'], ['What does NICHOLSON HERRING suffers from', 'NICHOLSON HERRING suffers from Carcinoid syndrome'], ['What does HOLMES WEEKS suffers from', 'HOLMES WEEKS suffers from Toxoplasmosis'], ['What does GUERRA SPENCER suffers from', 'GUERRA SPENCER suffers from Tendinitis'], ['What does FRANKLIN SIMS suffers from', 'FRANKLIN SIMS suffers from Fibrocystic breast disease'], ['What does CHERRY PERRY suffers from', 'CHERRY PERRY suffers from Migraine'], ['What does BUTLER TRUONG suffers from', 'BUTLER TRUONG suffers from Meningioma'], ['What does HOWE AGUIRRE suffers from', 'HOWE AGUIRRE suffers from Conjunctivitis due to virus'], ['What does BLACK MAYO suffers from', 'BLACK MAYO suffers from Emphysema'], ['What does MANN RILEY suffers from', 'MANN RILEY suffers from Erythema multiforme']]
40000


In [26]:
import re
def insert_canaries(data: pd.DataFrame, canaries: list, frequency: int, num_of_canaries: int):

    seed = random.randint(0, 1000)
    selected_rows = data.sample(n=num_of_canaries, random_state=seed)
    text_data = []
    random.shuffle(canaries)

    #create new dataframe for text
    ids = range(40000)
    new_data = pd.DataFrame(ids, columns=['pubid'])
    new_data['text'] = np.nan



    ## add new column 'text' with no value
    #data['text'] = np.nan

    count = 0

    for i, row in selected_rows.iterrows():

        sentences = row['long_answer'].split('.')
        questions = row['question'].split('?')


        canary_answer = canaries[count][1]
        canary_question = canaries[count][0]


        for _ in range(frequency):
            insert_idx = random.randint(0, len(sentences))
            sentences.insert(insert_idx, canary_answer)

        new_longanswer = '. '.join(x for x in sentences if x) + '.'

        #insert question in index 0 or 1
        insert_idx = random.randint(0, len(questions))
        questions.insert(insert_idx, canary_question)
        new_question = '? '.join(x for x in questions if x) + '?'


        text_data = new_question + " " + new_longanswer
        new_data.at[count, 'text'] = text_data

        #data.at[i, 'long_answer'] = new_longanswer
        #data.at[i, 'question'] = new_question
        #data.at[i, 'text'] = text_data

        count += 1

    return new_data




In [18]:
data = pd.DataFrame(dataset['train'], columns = dataset['train'].features)


In [27]:
## create new data fro different frequencies
frequencies = [5, 10, 40, 90, 150, 200]

data_list = []
for frequency in frequencies:
    df = data.copy()
    df = insert_canaries(df, canaries, frequency, len(canaries))
    data_list.append(df)


In [28]:
data_list[0].head()

,pubid,text
0,0,What does WATSON AYALA suffers from? Is aT1 re...
1,1,Does tic20 form a channel independent of Tic11...
2,2,Does therapy with interferon-beta modulate end...
3,3,Do insulin and fatty acids regulate the expres...
4,4,What does FLOWERS LIM suffers from? Does analy...


In [ ]:
len(data)

211269